In [ ]:
# test.py

# 1. Get user input data
age = int(input("Enter your age in years: "))
sex = input("Enter your sex (M/F): ").strip().upper()
weight_kg = float(input("Enter your weight in kilograms: "))
height_cm = float(input("Enter your height in centimeters: "))

# 2. BMR Function (Mifflin-St Jeor Equation)
def calculate_bmr(age, sex, weight_kg, height_cm):
    if sex == 'M':
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age + 5
    elif sex == 'F':
        bmr = 10 * weight_kg + 6.25 * height_cm - 5 * age - 161
    else:
        raise ValueError ("Sex must be 'M' or 'F'")
    return bmr

my_bmr = calculate_bmr(age, sex, weight_kg, height_cm)

# 3. BMR calculation
print("\nNow, let's consider your activity level:")
print("1. Sedentary (little or no exercise)")
print("2. Lightly active (light exercise/sports 1-3 days/week)")
print("3. Moderately active (moderate exercise/sports 3-5 days/week)")
activity_level = input("Enter your activity level (1, 2 or 3): ")

if activity_level == '1':
    activity_level = 1.2 # Sedentary
elif activity_level == '2':
    activity_level = 1.375 # Lightly active
elif activity_level == '3':
    activity_level = 1.55 # Moderately active
else:
    raise ValueError("Activity level must be 1, 2, or 3")

total_calories = my_bmr * activity_level

# 4. Output the result
print(f"\nBased on your info,  your Basal Metabololic Rate (BMR) is {my_bmr:.2f} calories/day.")
print(f"And your estimated Total Daily Energy Expenditure (TDEE) is: {total_calories:.2f} calories/day.")
print("\nRemember, this is an estimate! Consult a healthcare provider for personalised advice.")

In [ ]:
# calculations.py

def calculate_bmr(age, sex, weight_kg, height_cm):
   if sex == 'M':
      return (10 * weight_kg + 6.25 * height_cm - 5 * age + 5)
   elif sex == 'F':
      return (10 * weight_kg + 6.25 * height_cm - 5 * age - 161)
   else:
      raise ValueError ("Sex must be 'M' or 'F'")
   
def calculate_tdee(bmr, activity_level):
   return bmr * activity_level
  
def get_macro_recommendations(total_calories, medical_condition, weight_kg):
    
    protein_kcal, carb_kcal, fat_kcal = 0, 0, 0
    protein_g, carb_g, fat_g = 0, 0, 0

    if medical_condition == "general":
        # General guidelines for macronutrient distribution
        protein_percent = 0.20 # 20% of calories from protein
        carb_percent = 0.55    # 55% of calories from carbohydrates
        fat_percent = 0.25     # 25% of calories from fat

        protein_kcal = total_calories * protein_percent
        carb_kcal = total_calories * carb_percent
        fat_kcal = total_calories * fat_percent

    elif medical_condition == "diabetes":
        # General guidelines often focus on consistent carb intake,
        protein_percent = 0.20
        carb_percent = 0.50 
        fat_percent = 0.30

        protein_kcal = total_calories * protein_percent
        carb_kcal = total_calories * carb_percent
        fat_kcal = total_calories * fat_percent

    elif medical_condition == "renal_disease":
        # Critical: Protein restriction is common for non-dialysis CKD.
        if weight_kg == 0:
            print("Warning: Weight (kg) is zero for renal disease calculation. Cannot calculate protein.")
            return 0, 0, 0

        protein_g_target = 0.7 * weight_kg # g/kg body weight
        protein_kcal = protein_g_target * 4 # Convert grams to kcal

        # Distribute remaining calories from carbs and fats
        remaining_calories = total_calories - protein_kcal
        if remaining_calories < 0:
            # This can happen if TDEE is very low or protein target is relatively high
            print(f"Warning: Calculated protein ({protein_kcal:.0f} kcal) exceeds total calories ({total_calories:.0f} kcal). Adjusting.")
            protein_kcal = total_calories * 0.1 # Fallback to a minimal protein percentage
            remaining_calories = total_calories - protein_kcal

        carb_percent_of_remaining = 0.65
        fat_percent_of_remaining = 0.35

        carb_kcal = remaining_calories * carb_percent_of_remaining
        fat_kcal = remaining_calories * fat_percent_of_remaining

        # Convert all to grams
        protein_g = protein_kcal / 4
        carb_g = carb_kcal / 4
        fat_g = fat_kcal / 9

        if 'protein_g_target' not in locals(): # Check if original target was used
             protein_g = protein_kcal / 4
    else:
        print("Warning: Unknown medical condition. Using general guidelines.")
        return get_macro_recommendations(total_calories, "general", weight_kg)

    # Final conversion to grams
    protein_g = protein_kcal / 4
    carb_g = carb_kcal / 4
    fat_g = fat_kcal / 9

    return protein_g, carb_g, fat_g
   

In [5]:
# main_ver1.py

from calculations import calculate_bmr, calculate_tdee, get_macro_recommendations, calculate_bmi, classify_bmi, get_micronutrient_guidelines

def get_numeric_input(prompt, value_type=float, min_val=None, max_val=None, error_msg="Invalid input. Please enter a positive number."):
    while True:
        try:
            value = value_type(input(prompt))
            if value <= 0:
                raise ValueError("Value must be greater than zero.")
            if min_val is not None and value < min_val:
                raise ValueError(f"Value must be at least {min_val}.")
            if max_val is not None and value > max_val:
                raise ValueError(f"Value must be at most {max_val}.")
            if value == 999:
                exit ("Exiting the program.")
            return value
        except ValueError as e:
            print(e if str(e) else error_msg)

def get_sex_input():
    while True:
        sex = input("Enter your sex (M/F): ").upper()
        if sex in ['M', 'F']:
            return sex
        else:
            print("Invalid input. Please enter 'M' or 'F'.")

def get_activity_level():
    activity_levels = { # Changed to plural for better naming consistency
        1: 1.2,   # Sedentary
        2: 1.375, # Lightly active
        3: 1.55,  # Moderately active
        4: 1.725, # Very active
        5: 1.9    # Super active
    }

    print("\nNow, let's consider your activity level:")
    print("1. Sedentary (little or no exercise)")
    print("2. Lightly active (light exercise/sports 1-3 days/week)")
    print("3. Moderately active (moderate exercise/sports 3-5 days/week)")
    print("4. Very active (hard exercise/sports 6-7 days a week)")
    print("5. Super active (very hard exercise/sports & a physical job)")

    while True:
        try:
            level = int(input("Enter your activity level (1-5): "))
            if level in activity_levels:
                return activity_levels[level]
            else:
                print("Invalid input. Please enter a number between 1 and 5.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")

def get_medical_condition():
    conditions = {
        1: "general",
        2: "diabetes",
        3: "renal_disease",
        4: "hypertension", # Corrected typo from "hypertenstion"
        5: "heart_disease"
    }
    print("\nSelect your medical condition by number:")
    print("1. General/Healthy")
    print("2. Diabetes")
    print("3. Renal Disease (Non-dialysis, consult healthcare provider!)")
    print("4. Hypertension")
    print("5. Heart Disease")

    while True:
        try:
            choice = int(input("Enter your condition (1-5): ")) # Updated prompt based on added conditions
            if choice in conditions:
                return conditions[choice]
            else:
                print("Invalid choice. Please enter a number between 1-5.")
        except ValueError:
            print("Invalid input. Please enter a valid number.") # Updated error message

def get_weight_goal():
    print("\nWhat is your weight goal?")
    print("1. Maintain current weight")
    print("2. Lose weight (e.g., aiming for 0.5-1 kg/week)")
    print("3. Gain weight (e.g., aiming for 0.5-1 kg/week)")
    while True:
        try:
            goal_choice = int(input("Enter your choice (1-3): "))
            if goal_choice == 1:
                return "maintain"
            elif goal_choice == 2:
                return "lose"
            elif goal_choice == 3:
                return "gain"
            else:
                print("Invalid choice. Please enter 1, 2, or 3.")
        except ValueError:
            print("Invalid input. Please enter a valid number.")


if __name__ == "__main__":
    print("--- Nutrition Therapy Calculator ---")

    # Get all patient data with validation
    age = get_numeric_input("Enter your age in years: ", int, min_val=1, max_val=120)
    sex = get_sex_input()
    weight_kg = get_numeric_input("Enter your weight in kilograms (e.g., 70.5): ", min_val=20, max_val=300)
    height_cm = get_numeric_input("Enter your height in centimeters (e.g., 175.0): ", min_val=50, max_val=250)
    
    activity_factor = get_activity_level() 
    
    medical_condition = get_medical_condition()
    
    micronutrient_guidelines = get_micronutrient_guidelines(medical_condition)

    bmi = calculate_bmi(weight_kg, height_cm)
    bmi_category = classify_bmi(bmi)
    print(f"\nYour Body Mass Index (BMI): {bmi:.2f} ({bmi_category})")

    # Perform calculations
    try:
        bmr = calculate_bmr(age, sex, weight_kg, height_cm)
        tdee = calculate_tdee(bmr, activity_factor)
        
        weight_goal = get_weight_goal()
        
        recommended_calories = tdee 

        if weight_goal == "lose":
            # Typical deficit for 0.5-1 kg (1-2 lbs) per week
            recommended_calories = tdee - 500 
            # Add a safety net for dangerously low calories
            if recommended_calories < 1200 and sex == 'F':
                print("\nWarning: Calculated calories for weight loss are very low. Please consult a healthcare professional for safe guidance.")
                recommended_calories = 1200 # Set a floor
            elif recommended_calories < 1500 and sex == 'M':
                print("\nWarning: Calculated calories for weight loss are very low. Please consult a healthcare professional for safe guidance.")
                recommended_calories = 1500 # Set a floor
        elif weight_goal == "gain":
            # Typical surplus for 0.5-1 kg (1-2 lbs) per week
            recommended_calories = tdee + 500

        protein_g, carb_g, fat_g = get_macro_recommendations(tdee, medical_condition, weight_kg)

        # Display results
        print("\n--- Your Personalized Nutrition Plan ---")
        print(f"Basal Metabolic Rate (BMR): {bmr:.0f} kcal/day")
        print(f"Total Daily Energy Expenditure (TDEE): {tdee:.0f} kcal/day")
        
        # CONDITIONAL DISPLAY: Only show "Recommended Daily Calories" if a goal was set
        if weight_goal == "lose" or weight_goal == "gain":
            print(f"Recommended Daily Calories for {weight_goal.capitalize()}ing weight: {recommended_calories:.0f} kcal")

        print("\nMacronutrient Distribution:")
        print(f"Protein: {protein_g:.1f} grams")
        print(f"Carbohydrates: {carb_g:.1f} grams")
        print(f"Fats: {fat_g:.1f} grams")
        
        print("\nMicronutrient Guidelines:")
        if micronutrient_guidelines: # Check if there are guidelines to print
            for nutrient, guideline in micronutrient_guidelines.items():
                print(f"- {nutrient.replace('_', ' ').title()}: {guideline}")
        else:
            print("No specific micronutrient guidelines for your condition at this time.")

        # Additional warnings based on BMI
        if bmi_category == "Underweight":
            print("\nWarning: Your BMI suggests you might be underweight. Please consult a healthcare professional.")
        elif bmi_category.startswith("Obesity"): # Catches all Obesity classes
            print("\nWarning: Your BMI suggests you are in an obesity category. Please consult a healthcare professional for personalized guidance.")
        
        # Display medical condition-specific warnings/guidelines
        if medical_condition == "renal_disease":
            print("\n!!! IMPORTANT WARNING for Renal Disease Patients !!!")
            print("This tool provides *generalized* estimates. Renal disease nutrition is highly complex.")
            print("Fluid, potassium, and phosphorus restrictions are critical and individualized.")
            print("ALWAYS consult a nephrologist and a Registered Dietitian specializing in renal nutrition.")
            print("These calculations DO NOT replace professional medical advice.")
            print("Fluid restrictions: Individualized. Consult RD.") 

        if medical_condition == "diabetes":
            print("\nFiber Recommendation for Diabetes: Aim for 25-30 grams of fiber per day.")
            print("Meal Timing Suggestions for Diabetes:")
            print("- Aim for consistent meal and snack times daily.")
            print("- Distribute carbohydrate intake evenly throughout the day.")
            print("- Avoid skipping meals, especially if on medication that lowers blood sugar.")
        
        if medical_condition == "hypertension":
            print("\nWarning: For Hypertension, focus on a low-sodium diet (e.g., DASH diet). Consult a healthcare professional.")
        
        if medical_condition == "heart_disease":
            print("\nWarning: For Heart Disease, limit saturated fats to less than 7% of total calories. Consult a healthcare professional.")


        print("\n--- IMPORTANT DISCLAIMER ---")
        print("These are *estimates* and *examples* based on general guidelines.")
        print("Always consult a qualified healthcare professional (like a Registered Dietitian) for personalized nutrition therapy, especially for specific medical conditions.")

    # Error handling
    except ValueError as e:
        print(f"\nAn error occurred: {e}")
        print("Please check your inputs and try again.")
    except Exception as e:
        print(f"\nAn unexpected error occurred: {e}")

--- Nutrition Therapy Calculator ---

Now, let's consider your activity level:
1. Sedentary (little or no exercise)
2. Lightly active (light exercise/sports 1-3 days/week)
3. Moderately active (moderate exercise/sports 3-5 days/week)
4. Very active (hard exercise/sports 6-7 days a week)
5. Super active (very hard exercise/sports & a physical job)

Select your medical condition by number:
1. General/Healthy
2. Diabetes
3. Renal Disease (Non-dialysis, consult healthcare provider!)
4. Hypertension
5. Heart Disease

Your Body Mass Index (BMI): 24.86 (Normal weight)

What is your weight goal?
1. Maintain current weight
2. Lose weight (e.g., aiming for 0.5-1 kg/week)
3. Gain weight (e.g., aiming for 0.5-1 kg/week)

--- Your Personalized Nutrition Plan ---
Basal Metabolic Rate (BMR): 1740 kcal/day
Total Daily Energy Expenditure (TDEE): 2088 kcal/day

Macronutrient Distribution:
Protein: 53.9 grams
Carbohydrates: 304.3 grams
Fats: 72.8 grams

Micronutrient Guidelines:
- Sodium: Limit to < 2000

In [9]:
# main.py

from config_manager import SETTINGS
from logger_config import app_logger
from calculations import calculate_bmr, calculate_tdee, get_macro_recommendations, calculate_bmi, classify_bmi, get_micronutrient_guidelines

def get_numeric_input(prompt, value_type=float, min_val=None, max_val=None, error_msg="Invalid input. Please enter a positive number."):
    """
    Gets numeric input from the user with validation and logging.
    """
    while True:
        try:
            raw_user_input = input(prompt)
            value = value_type(raw_user_input)

            if value <= 0:
                raise ValueError("Value must be greater than zero.")
            if min_val is not None and value < min_val:
                raise ValueError(f"Value must be at least {min_val}.")
            if max_val is not None and value > max_val:
                raise ValueError(f"Value must be at most {max_val}.")
            if value == 999:
                app_logger.info("User initiated program exit with '999'.")
                exit("Exiting the program.")
            return value
        except ValueError as e:
            app_logger.warning(f"Invalid numeric input: {e}. User entered: '{raw_user_input}' for prompt: '{prompt.strip()}'")
            print(e if str(e) else error_msg)

def get_sex_input():
    """Gets validated sex input (M/F)."""
    while True:
        sex = input("Enter your sex (M/F): ").upper()
        if sex in ['M', 'F']:
            return sex
        else:
            app_logger.warning(f"Invalid sex input: '{sex}'. Please enter 'M' or 'F'.")
            print("Invalid input. Please enter 'M' or 'F'.")

def get_activity_level():
    """Gets validated activity level input and returns corresponding factor."""
    activity_levels = {
        1: 1.2,   # Sedentary
        2: 1.375, # Lightly active
        3: 1.55,  # Moderately active
        4: 1.725, # Very active
        5: 1.9    # Super active
    }

    print("\nNow, let's consider your activity level:")
    print("1. Sedentary (little or no exercise)")
    print("2. Lightly active (light exercise/sports 1-3 days/week)")
    print("3. Moderately active (moderate exercise/sports 3-5 days/week)")
    print("4. Very active (hard exercise/sports 6-7 days a week)")
    print("5. Super active (very hard exercise/sports & a physical job)")

    while True:
        try:
            level = int(input("Enter your activity level (1-5): "))
            if level in activity_levels:
                return activity_levels[level]
            else:
                app_logger.warning(f"Invalid activity level choice: {level}. Please enter a number between 1 and 5.")
                print("Invalid input. Please enter a number between 1 and 5.")
        except ValueError:
            app_logger.warning("Invalid activity level input. Non-numeric value entered.")
            print("Invalid input. Please enter a valid number.")

def get_medical_condition():
    """Gets validated medical condition input."""
    conditions = {
        1: "general",
        2: "diabetes",
        3: "renal_disease",
        4: "hypertension",
        5: "heart_disease"
    }
    
    print("\nSelect your medical condition by number:")
    print("1. General/Healthy")
    print("2. Diabetes")
    print("3. Renal Disease (Non-dialysis, consult healthcare provider!)")
    print("4. Hypertension")
    print("5. Heart Disease")

    while True:
        try:
            choice = int(input("Enter your choice (1-5): "))
            if choice in conditions:
                return conditions[choice]
            else:
                app_logger.warning(f"Invalid medical condition choice: {choice}. Please enter a number between 1-5.")
                print("Invalid choice. Please enter a number between 1-5.")
        except ValueError:
            app_logger.warning("Invalid medical condition input. Non-numeric value entered.")
            print("Invalid input. Please enter a valid number.")

def get_weight_goal():
    """Gets validated weight goal input."""
    print("\nWhat is your weight goal?")
    print("1. Maintain current weight")
    print("2. Lose weight (e.g., aiming for 0.5-1 kg/week)")
    print("3. Gain weight (e.g., aiming for 0.5-1 kg/week)")
    while True:
        try:
            goal_choice = int(input("Enter your choice (1-3): "))
            if goal_choice == 1:
                return "maintain"
            elif goal_choice == 2:
                return "lose"
            elif goal_choice == 3:
                return "gain"
            else:
                app_logger.warning(f"Invalid weight goal choice: {goal_choice}. Please enter 1, 2, or 3.")
                print("Invalid choice. Please enter 1, 2, or 3.")
        except ValueError:
            app_logger.warning("Invalid weight goal input. Non-numeric value entered.")
            print("Invalid input. Please enter a valid number.")

# NEW FUNCTION: save_nutrition_plan
def save_nutrition_plan(patient_data, calculated_results, file_path):
    """
    Saves the complete nutrition plan to a text file.
    """
    try:
        with open(file_path, 'w') as f:
            f.write("--- Personalized Nutrition Plan Report ---\n\n")

            f.write("1. Patient Information:\n")
            f.write(f"   Age: {patient_data['age']} years\n")
            f.write(f"   Sex: {patient_data['sex']}\n")
            f.write(f"   Weight: {patient_data['weight_kg']:.1f} kg\n")
            f.write(f"   Height: {patient_data['height_cm']:.1f} cm\n")
            f.write(f"   Activity Level: {patient_data['activity_level_description']} (Factor: {patient_data['activity_factor']})\n")
            f.write(f"   Medical Condition: {patient_data['medical_condition'].replace('_', ' ').title()}\n\n")

            f.write("2. Body Composition:\n")
            f.write(f"   Body Mass Index (BMI): {calculated_results['bmi']:.2f} ({calculated_results['bmi_category']})\n\n")

            f.write("3. Calorie and Macronutrient Recommendations:\n")
            f.write(f"   Basal Metabolic Rate (BMR): {calculated_results['bmr']:.0f} kcal/day\n")
            f.write(f"   Total Daily Energy Expenditure (TDEE): {calculated_results['tdee']:.0f} kcal/day\n")
            
            if calculated_results['weight_goal'] in ["lose", "gain"]:
                f.write(f"   Recommended Daily Calories for {calculated_results['weight_goal'].capitalize()}ing weight: {calculated_results['recommended_calories']:.0f} kcal\n")
            else:
                f.write(f"   Recommended Daily Calories (Maintain): {calculated_results['recommended_calories']:.0f} kcal\n")

            f.write("\n   Macronutrient Distribution:\n")
            f.write(f"     Protein: {calculated_results['protein_g']:.1f} grams\n")
            f.write(f"     Carbohydrates: {calculated_results['carb_g']:.1f} grams\n")
            f.write(f"     Fats: {calculated_results['fat_g']:.1f} grams\n\n")

            f.write("4. Micronutrient Guidelines:\n")
            if calculated_results['micronutrient_guidelines']:
                for nutrient, guideline in calculated_results['micronutrient_guidelines'].items():
                    f.write(f"   - {nutrient.replace('_', ' ').title()}: {guideline}\n")
            else:
                f.write("   No specific micronutrient guidelines for your condition at this time.\n")
            f.write("\n")

            f.write("5. Important Notes and Warnings:\n")
            warnings_list = []
            if calculated_results['bmi_category'] == "Underweight":
                warnings_list.append("Your BMI suggests you might be underweight. Please consult a healthcare professional.")
            elif calculated_results['bmi_category'].startswith("Obesity"):
                warnings_list.append(f"Your BMI suggests you are in an obesity category ({calculated_results['bmi_category']}). Please consult a healthcare professional for personalized guidance.")
            
            if patient_data['sex'] == 'F' and calculated_results['recommended_calories'] == SETTINGS['min_calories']['female']:
                warnings_list.append("Warning: Calculated calories for weight loss were very low for a female. Capped at minimum recommended (1200 kcal). Please consult a healthcare professional for safe guidance.")
            elif patient_data['sex'] == 'M' and calculated_results['recommended_calories'] == SETTINGS['min_calories']['male']:
                warnings_list.append("Warning: Calculated calories for weight loss were very low for a male. Capped at minimum recommended (1500 kcal). Please consult a healthcare professional for safe guidance.")

            # Medical condition specific warnings
            if patient_data['medical_condition'] == "renal_disease":
                warnings_list.append("\n!!! IMPORTANT WARNING for Renal Disease Patients !!!")
                warnings_list.append("This tool provides *generalized* estimates. Renal disease nutrition is highly complex.")
                warnings_list.append("Fluid, potassium, and phosphorus restrictions are critical and individualized.")
                warnings_list.append("ALWAYS consult a nephrologist and a Registered Dietitian specializing in renal nutrition.")
                warnings_list.append("These calculations DO NOT replace professional medical advice.")
                warnings_list.append("Fluid restrictions: Individualized. Consult RD.")
            if patient_data['medical_condition'] == "diabetes":
                warnings_list.append("Fiber Recommendation for Diabetes: Aim for 25-30 grams of fiber per day.")
                warnings_list.append("Meal Timing Suggestions for Diabetes:")
                warnings_list.append("- Aim for consistent meal and snack times daily.")
                warnings_list.append("- Distribute carbohydrate intake evenly throughout the day.")
                warnings_list.append("- Avoid skipping meals, especially if on medication that lowers blood sugar.")
            if patient_data['medical_condition'] == "hypertension":
                warnings_list.append("For Hypertension, focus on a low-sodium diet (e.g., DASH diet). Consult a healthcare professional.")
            if patient_data['medical_condition'] == "heart_disease":
                warnings_list.append("For Heart Disease, limit saturated fats to less than 7% of total calories. Consult a healthcare professional.")

            if warnings_list:
                for warning in warnings_list:
                    f.write(f"   - {warning}\n")
            else:
                f.write("   No specific warnings based on your inputs or conditions.\n")
            f.write("\n")

            f.write("--- IMPORTANT DISCLAIMER ---\n")
            f.write("These are *estimates* and *examples* based on general guidelines.\n")
            f.write("Always consult a qualified healthcare professional (like a Registered Dietitian) for personalized nutrition therapy, especially for specific medical conditions.\n")

        app_logger.info(f"Nutrition plan successfully saved to: {file_path}")
        print(f"\nNutrition plan saved successfully to: {file_path}")
    except IOError as e:
        app_logger.error(f"Error saving nutrition plan to {file_path}: {e}")
        print(f"Error: Could not save the nutrition plan. {e}")
    except Exception as e:
        app_logger.critical(f"An unexpected error occurred while saving the nutrition plan: {e}", exc_info=True)
        print(f"An unexpected error occurred while saving the nutrition plan. {e}")


if __name__ == "__main__":
    app_logger.info("Nutrition Therapy Calculator started.")

    print("--- Nutrition Therapy Calculator ---")

    # Dictionaries to store patient data and calculated results
    patient_data = {}
    calculated_results = {}

    try:
        # Get all patient data with validation
        patient_data['age'] = get_numeric_input("Enter your age in years: ", int, min_val=1, max_val=120)
        patient_data['sex'] = get_sex_input()
        patient_data['weight_kg'] = get_numeric_input("Enter your weight in kilograms (e.g., 70.5): ", min_val=20, max_val=300)
        patient_data['height_cm'] = get_numeric_input("Enter your height in centimeters (e.g., 175.0): ", min_val=50, max_val=250)
        
        patient_data['activity_factor'] = get_activity_level()
        # Get descriptive string for activity level for saving
        activity_descriptions = {1.2: "Sedentary", 1.375: "Lightly active", 1.55: "Moderately active", 1.725: "Very active", 1.9: "Super active"}
        patient_data['activity_level_description'] = next((desc for factor, desc in activity_descriptions.items() if factor == patient_data['activity_factor']), "Unknown")

        patient_data['medical_condition'] = get_medical_condition()
        
        calculated_results['micronutrient_guidelines'] = get_micronutrient_guidelines(patient_data['medical_condition'])

        calculated_results['bmi'] = calculate_bmi(patient_data['weight_kg'], patient_data['height_cm'])
        calculated_results['bmi_category'] = classify_bmi(calculated_results['bmi'])
        app_logger.info(f"Calculated BMI: {calculated_results['bmi']:.2f} ({calculated_results['bmi_category']})")
        print(f"\nYour Body Mass Index (BMI): {calculated_results['bmi']:.2f} ({calculated_results['bmi_category']})")

        # Perform calculations
        calculated_results['bmr'] = calculate_bmr(patient_data['age'], patient_data['sex'], patient_data['weight_kg'], patient_data['height_cm'])
        calculated_results['tdee'] = calculate_tdee(calculated_results['bmr'], patient_data['activity_factor'])

        app_logger.info(f"Calculated BMR: {calculated_results['bmr']:.0f} kcal/day, TDEE: {calculated_results['tdee']:.0f} kcal/day")

        # Get weight goal and adjust calories
        calculated_results['weight_goal'] = get_weight_goal()
        app_logger.info(f"User selected weight goal: {calculated_results['weight_goal']}")
        
        calculated_results['recommended_calories'] = calculated_results['tdee'] # Start with TDEE as the baseline

        # Use settings from config_manager
        if calculated_results['weight_goal'] == "lose":
            original_recommended_calories = calculated_results['recommended_calories'] - SETTINGS['calorie_adjustments']['weight_loss_deficit_kcal']
            
            if patient_data['sex'] == 'F' and original_recommended_calories < SETTINGS['min_calories']['female']:
                app_logger.warning(f"Female recommended calories ({original_recommended_calories:.0f}) below minimum ({SETTINGS['min_calories']['female']}). Capping at minimum.")
                print("\nWarning: Calculated calories for weight loss are very low. Please consult a healthcare professional for safe guidance.")
                calculated_results['recommended_calories'] = SETTINGS['min_calories']['female']
            elif patient_data['sex'] == 'M' and original_recommended_calories < SETTINGS['min_calories']['male']:
                app_logger.warning(f"Male recommended calories ({original_recommended_calories:.0f}) below minimum ({SETTINGS['min_calories']['male']}). Capping at minimum.")
                print("\nWarning: Calculated calories for weight loss are very low. Please consult a healthcare professional for safe guidance.")
                calculated_results['recommended_calories'] = SETTINGS['min_calories']['male']
            else:
                 calculated_results['recommended_calories'] = original_recommended_calories

        elif calculated_results['weight_goal'] == "gain":
            calculated_results['recommended_calories'] = calculated_results['tdee'] + SETTINGS['calorie_adjustments']['weight_gain_surplus_kcal']
        
        app_logger.info(f"Final recommended calories for '{calculated_results['weight_goal']}' goal: {calculated_results['recommended_calories']:.0f} kcal")

        calculated_results['protein_g'], calculated_results['carb_g'], calculated_results['fat_g'] = get_macro_recommendations(
            calculated_results['recommended_calories'], patient_data['medical_condition'], patient_data['weight_kg']
        )
        app_logger.info(f"Macronutrients calculated: P:{calculated_results['protein_g']:.1f}g, C:{calculated_results['carb_g']:.1f}g, F:{calculated_results['fat_g']:.1f}g")

        # Display results
        print("\n--- Your Personalized Nutrition Plan ---")
        print(f"Basal Metabolic Rate (BMR): {calculated_results['bmr']:.0f} kcal/day")
        print(f"Total Daily Energy Expenditure (TDEE): {calculated_results['tdee']:.0f} kcal/day")
        
        if calculated_results['weight_goal'] in ["lose", "gain"]:
            print(f"Recommended Daily Calories for {calculated_results['weight_goal'].capitalize()}ing weight: {calculated_results['recommended_calories']:.0f} kcal")
        else:
            print(f"Recommended Daily Calories (Maintain): {calculated_results['recommended_calories']:.0f} kcal") # Explicitly show maintain

        print("\nMacronutrient Distribution:")
        print(f"Protein: {calculated_results['protein_g']:.1f} grams")
        print(f"Carbohydrates: {calculated_results['carb_g']:.1f} grams")
        print(f"Fats: {calculated_results['fat_g']:.1f} grams")
        
        print("\nMicronutrient Guidelines:")
        if calculated_results['micronutrient_guidelines']:
            app_logger.info("Micronutrient guidelines provided based on medical condition.")
            for nutrient, guideline in calculated_results['micronutrient_guidelines'].items():
                print(f"- {nutrient.replace('_', ' ').title()}: {guideline}")
        else:
            app_logger.info("No specific micronutrient guidelines for selected condition (defaulting to general if no match).")
            print("No specific micronutrient guidelines for your condition at this time.")

        # Additional warnings based on BMI
        if calculated_results['bmi_category'] == "Underweight":
            app_logger.warning("BMI is Underweight, advising consultation.")
            print("\nWarning: Your BMI suggests you might be underweight. Please consult a healthcare professional.")
        elif calculated_results['bmi_category'].startswith("Obesity"):
            app_logger.warning(f"BMI is in Obesity category ({calculated_results['bmi_category']}), advising consultation.")
            print("\nWarning: Your BMI suggests you are in an obesity category. Please consult a healthcare professional for personalized guidance.")
        
        # Display medical condition-specific warnings/guidelines
        if patient_data['medical_condition'] == "renal_disease":
            app_logger.warning("Renal disease selected. Displaying critical specific warnings.")
            print("\n!!! IMPORTANT WARNING for Renal Disease Patients !!!")
            print("This tool provides *generalized* estimates. Renal disease nutrition is highly complex.")
            print("Fluid, potassium, and phosphorus restrictions are critical and individualized.")
            print("ALWAYS consult a nephrologist and a Registered Dietitian specializing in renal nutrition.")
            print("These calculations DO NOT replace professional medical advice.")
            print("Fluid restrictions: Individualized. Consult RD.") 

        if patient_data['medical_condition'] == "diabetes":
            app_logger.info("Diabetes selected. Displaying specific recommendations/warnings.")
            print("\nFiber Recommendation for Diabetes: Aim for 25-30 grams of fiber per day.")
            print("Meal Timing Suggestions for Diabetes:")
            print("- Aim for consistent meal and snack times daily.")
            print("- Distribute carbohydrate intake evenly throughout the day.")
            print("- Avoid skipping meals, especially if on medication that lowers blood sugar.")
        
        if patient_data['medical_condition'] == "hypertension":
            app_logger.info("Hypertension selected. Displaying specific recommendations/warnings.")
            print("\nWarning: For Hypertension, focus on a low-sodium diet (e.g., DASH diet). Consult a healthcare professional.")
        
        if patient_data['medical_condition'] == "heart_disease":
            app_logger.info("Heart disease selected. Displaying specific recommendations/warnings.")
            print("\nWarning: For Heart Disease, limit saturated fats to less than 7% of total calories. Consult a healthcare professional.")

        print("\n--- IMPORTANT DISCLAIMER ---")
        print("These are *estimates* and *examples* based on general guidelines.")
        print("Always consult a qualified healthcare professional (like a Registered Dietitian) for personalized nutrition therapy, especially for specific medical conditions.")
        app_logger.info("Nutrition Therapy Calculator finished successfully.")

        # Ask user to save the plan
        while True: # Keep looping until a valid choice is made
            save_choice = input("\nDo you want to save this nutrition plan to a file? (yes/no): ").lower()
            if save_choice == 'yes':
                while True: # This inner loop is for valid filename, already robust
                    filename = input("Enter filename (e.g., my_plan.txt): ")
                    if filename.strip(): # Ensure filename is not empty
                        save_nutrition_plan(patient_data, calculated_results, filename)
                        break # Break from filename loop
                    else:
                        print("Filename cannot be empty. Please enter a valid filename.")
                break # Break from save_choice loop (since 'yes' was handled)
            elif save_choice == 'no':
                app_logger.info("User chose not to save the nutrition plan.")
                print("Nutrition plan not saved.")
                break # Break from save_choice loop (since 'no' was handled)
            else: 
                app_logger.warning(f"Invalid save choice: {save_choice}. Expected 'yes' or 'no'.")
                print("Invalid choice. Please enter 'yes' or 'no'.") # Re-prompt due to outer loop

    except ValueError as e:
        app_logger.error(f"A ValueError occurred during program execution: {e}", exc_info=True)
        print(f"\nAn error occurred: {e}")
        print("Please check your inputs and try again.")
    except Exception as e:
        app_logger.critical(f"An unexpected critical error occurred: {e}", exc_info=True)
        print(f"\nAn unexpected error occurred: {e}")

--- Nutrition Therapy Calculator ---

Now, let's consider your activity level:
1. Sedentary (little or no exercise)
2. Lightly active (light exercise/sports 1-3 days/week)
3. Moderately active (moderate exercise/sports 3-5 days/week)
4. Very active (hard exercise/sports 6-7 days a week)
5. Super active (very hard exercise/sports & a physical job)

Select your medical condition by number:
1. General/Healthy
2. Diabetes
3. Renal Disease (Non-dialysis, consult healthcare provider!)
4. Hypertension
5. Heart Disease

Your Body Mass Index (BMI): 28.09 (Overweight)

What is your weight goal?
1. Maintain current weight
2. Lose weight (e.g., aiming for 0.5-1 kg/week)
3. Gain weight (e.g., aiming for 0.5-1 kg/week)

Enter diabetes type:
1. Type 1
2. Type 2
3. Gestational

--- Your Personalized Nutrition Plan ---
Basal Metabolic Rate (BMR): 1840 kcal/day
Total Daily Energy Expenditure (TDEE): 2208 kcal/day
Recommended Daily Calories for Loseing weight: 1708 kcal

Macronutrient Distribution:
Prote